# Script to look at incorrectly identified test images

In [ ]:
# Imports
import sys  
sys.path.insert(0, '../visual_home_finder')

import imp
import config, paths, utilities

imp.reload(config)
imp.reload(utilities)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
# Load our home-style feature model
home_model = utilities.home_model()

# Initialize validation/test data generator object
batch_size = len(list(paths.list_images(config.TEST_PATH)))
test_aug = ImageDataGenerator(featurewise_center = True)
test_aug.mean = config.IMG_MEAN
test_gen = test_aug.flow_from_directory(
    config.TEST_PATH,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=batch_size)

In [ ]:
# Reset test generator and used trained model to make predictions
print("Evaluating test cases...")
test_gen.reset()
pred_indices_raw = home_model.predict(test_gen)
# For each image, find the class
pred_indices = np.argmax(pred_indices_raw, axis=1)
# Classification Report
print(classification_report(test_gen.classes, pred_indices, 
                 target_names=test_gen.class_indices.keys()))

In [ ]:
batchX, batchy = test_gen.next()

In [ ]:
actual_class = 0
confused_with_class = 4
class_args = np.ravel(np.argwhere(test_gen.classes == actual_class))
confusion_args = np.ravel(np.argwhere(pred_indices[class_args]==confused_with_class))

In [ ]:
for ii in np.ravel(confusion_args):
    plt.figure()
    plt.title(pred_indices_raw[class_args[ii],:])
    plt.imshow((batchX[class_args[ii],:,:,:] + config.IMG_MEAN)/256)